# Amazon VPC Security
There are various options available for controlling and enhancing the security of resources within a cloud infrastructure. This notebook will explore some of these options, including fundamental VPC security concepts such as Security Groups and nACLs (network access control lists), as well as more sophisticated network protection solutions such as AWS Network Firewall and AWS Web Application Firewall (WAF).

## Security Groups
>*Security Groups* are collections of rules which control the network traffic in and out of the **instances** they are associated with, such as EC2 or RDS instances. 

These are the important characteristics of Security Groups you need to know:

- When you create a VPC, a Security Group is created by default, which is set to allow all traffic in and out of the instances it is associated with

- They can only be made to allow traffic, not deny

- They are *stateful*, which means if a connection is allowed, it will continue to exist bidirectionally until it is closed. In other words, security groups retain the state of a connection throughout the interaction.

### Security Group Rules
Security Group rules are created as either inbound or outbound rules, where inbound traffic comes from an external source to the instance, and outbound traffic comes from the instance to an external destination. Each rule specifies the following attributes:

- **Type:** This is the protocol you want to allow in or out of your instance. For example, some common protocols to choose are HTTP/HTTPS for web traffic, SSH for secure communication, or TCP. 

- **Protocol:** This will be greyed out and filled in for you, unless you select *Custom Protocol* in the Type dropdown list, in which case you can manually type the protocol yourself
- **Port range:** A specific port number or range of port numbers traffic is allowed to access. Again this might be auto-populated depending on what protocol type you chose. 
- **Source/Destination:** For inbound rules, the source of the traffic. For outbound rules, the destination where you want the traffic to go. This can be either your own IP address, any IP address, or a custom IP address which you then enter in the next box.
- **Description:** An optional description of what the rule is and why it is there

### Hands-On: Configuring Security Groups for a Web Server and Database
In the following common scenario, a website is being hosted on a web server, which communicates with a database server to store data such as customer, product, or sales information. We need to set up the security groups for both web server and database server so that they can communicate with each other securely. A typical architecture for this set up would be for the web server to be hosted in a public subnet and the database in a private subnet.

#### Creating a Security Group for the Web Server

1. Log in to the AWS management console and navigate to VPC using the search bar, then select *Security groups* on the sidebar, then *Create security group*

2. The first inbound rule here is to allow SSH traffic from your own public IP address. This is so that you can connect to the instance yourself.

- The second rule allows web traffic from any IP address to reach the instance. This is essential for people on the internet to be able to access the website hosted by the instance. 


<p align="center"> <img src="images/web-server-SG-inbound.png" height="225" width="1610"/> </p> 


<p align="center"> <img src="images/web-server-SG-outbound.png" height="178" width="1608"/> </p>

- Only one outbound rule is needed: to allow traffic from the web server to reach any IP address.

#### Security Groups for the Database

<p align="center"> <img src="images/database-SG-inbound.png" height="190" width="1615"/> </p>

- This rule allows TCP traffic from the web server security group to port 5432 on the database server. This is the default port used by PostgreSQL.

<p align="center"> <img src="images/database-SG-outbound.png" height="185" width="1605"/> </p>

- The only outbound rule needed for the database is one to allow it to receive software updates. 

## NACLs (Network Access Control Lists)
>*NACLs* are collections of rules which control the network traffic in and out of the **subnets** they are associated with.

These are the important characteristics of NACLs you need to know:

- Each subnet must be associated with one NACL. New subnets get a default NACL, which accepts all inbound and outbound traffic.

- NACLs can be customised to allow or deny traffic

- NACL rules are numbered. Rules are evaluated in order of their rule number, so lower numbers get a higher precedence. The first matching rule decides whether to allow or deny the traffic.
For example, traffic will be allowed in to an instance if there is a rule of number 100 which allows it, even if there is another rule of number 200 which denies it. 

- They are *stateless*, which means that every request and response in and out of a subnet is checked against the rules exclusively

#### NACL Rules
NACL rules contain all of the same rules as Security Group rules, with the addition of two extra fields:

- **Rule Number:** The order of precedence for which rules to evaluate first. AWS recommends you number rules using increments of 100, so that you can add more in between if necessary. 

- **Allow/Deny:** Whether the rule is to allow or deny traffic

Also, for NACLs the **Source** field is not a drop down list, you must enter the CIDR block manually.

#### NACLs and Ephemeral Ports
For an instance and a target to establish a connection they must use ports: the instance connects to a *defined port* on the target, and expects a response on an *ephemeral port*. This is important to remember when creating NACL rules, since NACLs are stateless. 

When creating rules for inbound traffic we need to open the required ephemeral port range so that a response can be received by the instance. The port range varies depending on the instance's operating system: Linux machines use 32768-61000, most recent Windows versions use 49152-65535, for example.

The image below is an example of adding outbound rules to a Security Group. In this particular case the rule is set to allow HTTP traffic out to any IPv4 address. HTTP automatically uses a TCP connection and comes from port 80 of the instance.

<p align="center"> <img src="images/SGs.png" height="267" width="1733"/> </p>

### Hands-On: Creating and Configuring Security Groups and NACLs [WIP]
In this Hands-On we are going to set up the security so that two EC2 instances in separate subnets can communicate with eachother. This will involve creating both instances, then creating and configuring Security Group and NACL inbound and outbound rules for each. We will be using only free tier resources. 

#### Step 1: Create and Configure The EC2 instances
- Log in to your AWS management console and navigate to EC2 using the search bar. Make sure the region is set to Ireland.

- Press *Launch instance*
- Enter a name for your instance. Make sure the Instance type is *t2.micro* and says *Free tier eligible*.
- Create new key pair, making sure it is a `.pem` file. It will automatically download when you press *Create key pair*, make sure it is somewhere safe on your local computer.
- In *Network settings* select a VPC and public subnet (you can use one from a previous Hands-on)
- Select *Enable* for *Auto-assign public IP*. In order to connect to our EC2s from the AWS console they need to have public IP addresses assigned to them.
- Make sure that *Create security group* is selected, but *Allow SSH traffic from* is deselected (we are going to manually configure this in a later step)
- Leave everything else the same then press *Launch instance*
- Complete this whole step once more to create another EC2 instance, making sure to call it something different and placing it in a separate subnet

#### Step 2: Ensure public subnets are routed correctly
- If you are using public subnets from a previous Hands-on then double check you still have an Internet Gateway routed to the subnet

- If you have created new subnets for this exercise, then you will have to also create an Internet Gateway to make the subnets public. Revisit (this)[] notebook to find out how.

#### Step 3: Create Security Group for instance-1
- Navigate to the VPCs console, then Security Groups on sidebar
**Inbound Rules**

**Outbound Rules**

#### Step 4: Create NACL for instance-1
**Inbound Rules**

**Outbound Rules**

#### Step 5: Create Security Group for instance-2
**Inbound Rules**

**Outbound Rules**

#### Step 5: Create NACL for instance-2
**Inbound Rules**

**Outbound Rules**

### Step 6: Test the set up


### Firewalls  
A *firewall* is a network security device which monitors incoming and outgoing traffic and decides whether to allow or block specific traffic based on a defined set of security rules. This section outlines two of the main firewall services available for AWS.

#### AWS Network Firewall
>*AWS Network Firewall* is an intrusion detection and prevention service for Amazon VPCs.

It is made up of the following components:

- **Rule Group:** A reusable collection of criteria which decide whether packets in a traffic flow are allowed to pass or drop. For example, if the traffic came from a particular range of IP addresses. Rule groups are either stateless or stateful: stateless rule groups evaluate packets in isolation, while stateful rule groups evaluate them in the context of their traffic flow. 

- **Firewall Policy:** A reusable collection of Rule Groups, along with some policy-level behaviour such as what happens if traffic breaks midstream, and default actions for both stateless and stateful traffic.
- **Firewall:** Connects the Firewall Policy to the VPC. Each Firewall requires at least one Firewall Policy. The Firewall additionally defines settings like how to monitor and log network traffic information.

#### AWS Web Application Firewall (WAF)
>*AWS WAF* is a network protection and monitoring service specifically designed for HTTP/HTTPS traffic to and from web application resources. 

It works by defining traffic rules and then associating them with web application resources. A rule describes traffic patterns to look for in requests and to specify the actions to take on matching requests. These actions can be:

- Allow the requests to go to the protected resource for processing and response
- Block the requests
- Count the requests
- Run CAPTCHA or challenge checks against requests to verify human users and standard browser use

AWS WAF is made up of the following components:

- **Rules:** Rules contain statements that define the traffic inspection criteria, then the action to take if a request meets the criteria. For example:
    - Defining IP addresses or ranges which requests come from
    - The country or location which requests originate from
    - You can check for specific known malicious request attributes such as the query string, the request header value or body values

- **Rule Groups:** Reusable collections of rules. You can create your own rule groups or use predefined ones from the AWS Marketplace.
- **Web ACLs (Access Control Lists):** Collections of rule groups, where you decide in which order the rules and rule groups should be evaluated

## Key Takeaways
- Security Groups allow you to control network traffic to and from specific instances. They are stateful, and can only be made to allow traffic, not deny.

- NACLs allow you to control network traffic at the subnet level. They are stateless and can allow or deny traffic.
- SG and NACL rules are defined as either inbound or outbound, specifying the protocol type, port range, source/destination, and a description. Additionally, NACL rules are numbered, and are evaluated in order of their numbers.
- AWS Network Firewall is an intrusion detection and monitoring service for VPCs. It consists of traffic rules, grouped together into Rule Groups, which are then assigned to a Firewall Policy.
- AWS WAF is a network detection and monitoring service for HTTP/HTTPS traffic to and from web application resources. A WAF consists of a web ACL, which is a collection of rule groups. 